In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import polars as pl # data processing, CSV files

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv
/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv
/kaggle/input/llm-detect-ai-generated-text/test_essays.csv
/kaggle/input/llm-detect-ai-generated-text/train_essays.csv
/kaggle/input/llm-generated-essays/ai_generated_train_essays.csv
/kaggle/input/llm-generated-essays/ai_generated_train_essays_gpt-4.csv


# **Test Essays**

In [2]:
test_essays_df = pl.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')
test_essays_df

id,prompt_id,text
str,i64,str
"""0000aaaa""",2,"""Aaa bbb ccc."""
"""1111bbbb""",3,"""Bbb ccc ddd."""
"""2222cccc""",4,"""CCC ddd eee."""


# **Train Prompts**

In [3]:
train_prompts_df = pl.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv')
train_prompts_df

prompt_id,prompt_name,instructions,source_text
i64,str,str,str
0,"""Car-free citie…","""Write an expla…","""# In German Su…"
1,"""Does the elect…","""Write a letter…","""# What Is the …"


In [4]:
train_prompts_df['prompt_name'][0]

'Car-free cities'

In [5]:
train_prompts_df['instructions'][0]

'Write an explanatory essay to inform fellow citizens about the advantages of limiting car usage. Your essay must be based on ideas and information that can be found in the passage set. Manage your time carefully so that you can read the passages; plan your response; write your response; and revise and edit your response. Be sure to use evidence from multiple sources; and avoid overly relying on one source. Your response should be in the form of a multiparagraph essay. Write your essay in the space provided.'

In [6]:
train_prompts_df['source_text'][0]

'# In German Suburb, Life Goes On Without Cars by Elisabeth Rosenthal\n\n1 VAUBAN, Germany—Residents of this upscale community are suburban pioneers, going where few soccer moms or commuting executives have ever gone before: they have given up their cars.\n\n2 Street parking, driveways and home garages are generally forbidden in this experimental new district on the outskirts of Freiburg, near the French and Swiss borders. Vauban’s streets are completely “car-free”—except the main thoroughfare, where the tram to downtown Freiburg runs, and a few streets on one edge of the community. Car ownership is allowed, but there are only two places to park—large garages at the edge of the development, where a car-owner buys a space, for $40,000, along with a home.\n\n3 As a result, 70 percent of Vauban’s families do not own cars, and 57 percent sold a car to move here. “When I had a car I was always tense. I’m much happier this way,” said Heidrun Walter, a media trainer and mother of two, as she 

In [7]:
train_prompts_df['prompt_name'][1]

'Does the electoral college work?'

In [8]:
train_prompts_df['instructions'][1]

'Write a letter to your state senator in which you argue in favor of keeping the Electoral College or changing to election by popular vote for the president of the United States. Use the information from the texts in your essay. Manage your time carefully so that you can read the passages; plan your response; write your response; and revise and edit your response. Be sure to include a claim; address counterclaims; use evidence from multiple sources; and avoid overly relying on one source. Your response should be in the form of a multiparagraph essay. Write your response in the space provided.'

In [9]:
train_prompts_df['source_text'][1]

"# What Is the Electoral College? by the Office of the Federal Register\n\n1 The Electoral College is a process, not a place. The founding fathers established it in the Constitution as a compromise between election of the President by a vote in Congress and election of the President by a popular vote of qualified citizens.\n\n2 The Electoral College process consists of the selection of the electors, the meeting of the electors where they vote for President and Vice President, and the counting of the electoral votes by Congress.\n\n3 The Electoral College consists of 538 electors. A majority of 270 electoral votes is required to elect the President. Your state’s entitled allotment of electors equals the number of members in its Congressional delegation: one for each member in the House of Representatives plus two for your Senators. . . .\n\n4 Under the 23rd Amendment of the Constitution, the District of Columbia is allocated 3 electors and treated like a state for purposes of the Electo

# **Train Essays**

In [10]:
train_essays_df = pl.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_essays.csv')
train_essays_df

id,prompt_id,text,generated
str,i64,str,i64
"""0059830c""",0,"""Cars. Cars hav…",0
"""005db917""",0,"""Transportation…",0
"""008f63e3""",0,"""""America's lov…",0
"""00940276""",0,"""How often do y…",0
"""00c39458""",0,"""Cars are a won…",0
"""00da8c32""",1,"""The electrol c…",0
"""011dc2bc""",1,"""Dear state sen…",0
"""01448434""",0,"""Fellow citizen…",0
"""01c6e176""",1,"""""It's official…",0


In [11]:
train_essays_df['text'][0]

'Cars. Cars have been around since they became famous in the 1900s, when Henry Ford created and built the first ModelT. Cars have played a major role in our every day lives since then. But now, people are starting to question if limiting car usage would be a good thing. To me, limiting the use of cars might be a good thing to do.\n\nIn like matter of this, article, "In German Suburb, Life Goes On Without Cars," by Elizabeth Rosenthal states, how automobiles are the linchpin of suburbs, where middle class families from either Shanghai or Chicago tend to make their homes. Experts say how this is a huge impediment to current efforts to reduce greenhouse gas emissions from tailpipe. Passenger cars are responsible for 12 percent of greenhouse gas emissions in Europe...and up to 50 percent in some carintensive areas in the United States. Cars are the main reason for the greenhouse gas emissions because of a lot of people driving them around all the time getting where they need to go. Article

### **Only three essays have been generated using Large Language Models (LLM).**

In [12]:
train_essays_df['generated'].sum()

3

### **Adding AI generated essays to the dataset...**

In [13]:
GPT_3_5_essays = pl.read_csv('/kaggle/input/llm-generated-essays/ai_generated_train_essays.csv')
GPT_3_5_essays

id,prompt_id,text,generated
str,i64,str,i64
"""d429f032""",0,"""Advantages of …",1
"""1ce279be""",0,"""Advantages of …",1
"""c9595213""",0,"""Limiting car u…",1
"""f2266d87""",0,"""The passages p…",1
"""eeace4bd""",0,"""Title: The Adv…",1
"""354fdce0""",0,"""Advantages of …",1
"""6eaa842f""",0,"""The advantages…",1
"""a36a04d5""",0,"""Limiting car u…",1
"""c9d5567f""",0,"""Advantages of …",1


In [14]:
GPT_4_essays = pl.read_csv('/kaggle/input/llm-generated-essays/ai_generated_train_essays_gpt-4.csv')
GPT_4_essays

id,prompt_id,text,generated
str,i64,str,i64
"""d62377cd""",0,"""Title: The Ben…",1
"""d721137f""",0,"""Title: The Ben…",1
"""afc4d9ce""",0,"""Title: The Adv…",1
"""1dd6eddd""",0,"""Title: The Ben…",1
"""9baa7169""",0,"""Title: Advanta…",1
"""f75b6750""",0,"""Title: Advanta…",1
"""509e6e0c""",0,"""Title: The Adv…",1
"""6ee1ad8b""",0,"""Title: Advanta…",1
"""f405dd9e""",0,"""Title: The Adv…",1


In [15]:
train_essays_df = pl.concat([train_essays_df, GPT_3_5_essays, GPT_4_essays], rechunk=True)
train_essays_df

id,prompt_id,text,generated
str,i64,str,i64
"""0059830c""",0,"""Cars. Cars hav…",0
"""005db917""",0,"""Transportation…",0
"""008f63e3""",0,"""""America's lov…",0
"""00940276""",0,"""How often do y…",0
"""00c39458""",0,"""Cars are a won…",0
"""00da8c32""",1,"""The electrol c…",0
"""011dc2bc""",1,"""Dear state sen…",0
"""01448434""",0,"""Fellow citizen…",0
"""01c6e176""",1,"""""It's official…",0


In [16]:
train_essays_df.describe()

describe,id,prompt_id,text,generated
str,str,f64,str,f64
"""count""","""2078""",2078.0,"""2078""",2078.0
"""null_count""","""0""",0.0,"""0""",0.0
"""mean""",null,0.490857,null,0.338306
"""std""",null,0.500037,null,0.473247
"""min""","""00206890""",0.0,""""" All of our d…",0.0
"""25%""",null,0.0,null,0.0
"""50%""",null,0.0,null,0.0
"""75%""",null,1.0,null,1.0
"""max""","""ffe1ca0d""",1.0,"""when looking a…",1.0


In [17]:
# Shuffle the DataFrame
train_essays_df = train_essays_df.sample(fraction=1.0, shuffle=True, seed=42)

# Perform an 80:20 split
test_size = 420
test_df, train_df = train_essays_df.head(test_size), train_essays_df.tail(-test_size)

## **Generated: 1 => LLM (AI) generated**
## **Generated: 0 => Student written**

In [18]:
Student_essays = train_essays_df.filter(train_essays_df['generated'] == 0)
LLM_essays = train_essays_df.filter(train_essays_df['generated'] == 1)
LLM_essays

id,prompt_id,text,generated
str,i64,str,i64
"""df69e6ca""",1,"""Dear Senator […",1
"""68ad0312""",0,"""Title: The Ben…",1
"""bad69fc1""",0,"""Advantages of …",1
"""b4e6f8e6""",1,"""[Your Name] [Y…",1
"""d4fcfd4a""",0,"""Limiting car u…",1
"""15c27f02""",1,"""Dear Senator, …",1
"""d429f032""",0,"""Advantages of …",1
"""d01b5128""",0,"""Limiting car u…",1
"""0d1a2bb2""",1,"""Dear Senator, …",1


In [19]:
random_Student_essay = Student_essays.sample(n=1)
random_Student_essay

id,prompt_id,text,generated
str,i64,str,i64
"""c168b452""",0,"""The black smok…",0


In [20]:
random_LLM_essay = LLM_essays.sample(n=1)
random_LLM_essay

id,prompt_id,text,generated
str,i64,str,i64
"""8ee42515""",0,"""Limiting car u…",1


In [21]:
import nltk

In [22]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [23]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [24]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [25]:
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet

from collections import Counter

In [26]:
def preprocess_text(text):
    # Tokenize the text to words list
    words_list = word_tokenize(text)
    
    # Remove non-alphabetic characters and convert to lower-case
    real_words = [word.lower() for word in words_list if word.isalpha()]
    
    # Remove the StopWords
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in real_words if word not in stop_words]
    
    # Appying Lemmatization to each word
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]
    
    # Checking whether word has a synset in WorNet.
    filtered_words = [word for word in lemmatized_words if len(wordnet.synsets(word)) > 0]
    
    return filtered_words

def word_frequency_dictonary(words_list):
    word_counts = Counter(words_list)
    word_dict = dict(word_counts)
    return dict(sorted(word_dict.items(), key=lambda item: item[1]))

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    words = word_tokenize(text)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized_words)

def stem_text(text):
    stemmer = PorterStemmer()
    words = word_tokenize(text)
    stemmed_words = [stemmer.stem(word) for word in words]
    return ' '.join(stemmed_words)

def is_word_in_dictionary(word):
    try:
        synsets = wordnet.synsets(word)
        return len(synsets) > 0
    except Exception as e:
        print(f"Error: {e}")
        return False
    
def word_frequency_dict(text):
    words = word_tokenize(text)
    word_counts = Counter(words)
    return dict(word_counts)

def sort_dict_frequency(dictonary):
    return dict(sorted(dictonary.items(), key=lambda item: item[1]))

def sort_dict_words(dictonary):
    return dict(sorted(dictonary.items()))

In [27]:
EssayText = remove_stopwords(train_essays_df['text'][0])
EssayText

"Presidenas , ahey ahe mosa powerful people ahe Uniaed Saaaes America . ahe people make ahe aough calls ahe masses ao make sure ahe benefias ahe majoriay mea ahe needs ahe many oua weigh ahe needs ahe . people ahink ahaa ahe way presidenas elecaed aoday noa pracaice meahods bua people . Presidenas elecaed using awo processes , ahose processes popular voae Elecaoral College . Elecaoral College noa pracaical book ahey don'a spend aime ahe places ahaa ahey know ahey going ao win noa going ao Elecaoral College noa majoriay . Presidenas like supersaars ahe poliaical world bua ahey don'a spend aime ahe places ahey know ahey win lose . Source 2 saaaes `` Candidaaes dona spend ahe aime saaaes ahey know ahey chance winning . '' candidaaes ahe presidenaial elecaion '' even ary ao sway ahe saaaes ahaa eiaher ahey know aren'a going ao voae ahem ahe opponena already ahere gave ahem idea ceaain maaaers . Voaers usally ahoughaful ahey dona know ao voae ahe lisaen ao boah sides , bua ahere also ahe pr

In [28]:
#!dir /usr/share/nltk_data

In [29]:
#!dir /usr/share/nltk_data/corpora

In [30]:
#!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

In [31]:
try:
    LemmatizeText = lemmatize_text(EssayText)
    print(LemmatizeText)
except:
    !unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/
    LemmatizeText = lemmatize_text(EssayText)
    print(LemmatizeText)

Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

In [32]:
StemText = stem_text(EssayText)
StemText

"presidena , ahey ahe mosa power peopl ahe unia saaae america . ahe peopl make ahe aough call ahe mass ao make sure ahe benefia ahe majoriay mea ahe need ahe mani oua weigh ahe need ahe . peopl ahink ahaa ahe way presidena eleca aoday noa pracaic meahod bua peopl . presidena eleca use awo process , ahos process popular voae elecaor colleg . elecaor colleg noa pracaic book ahey don'a spend aim ahe place ahaa ahey know ahey go ao win noa go ao elecaor colleg noa majoriay . presidena like supersaar ahe poliaic world bua ahey don'a spend aim ahe place ahey know ahey win lose . sourc 2 saaae `` candidaa dona spend ahe aim saaae ahey know ahey chanc win . '' candidaa ahe presidenai elecaion '' even ari ao sway ahe saaae ahaa eiaher ahey know aren'a go ao voae ahem ahe opponena alreadi aher gave ahem idea ceaain maaaer . voaer usal ahougha ahey dona know ao voae ahe lisaen ao boah side , bua aher also ahe problem majoriay place noa oaher . majoriay also play impaca ahe voae ahe presid . look 

## **Making master dictonary (frequency of each word) Student dataset**

In [33]:
# Group by 'prompt_id' and concatenate all text values within each group into a single string
Student_essays = Student_essays.group_by('prompt_id').agg(
    pl.col('text').str.concat()
)
Student_essays

prompt_id,text
i64,str
0,"""""But America's…"
1,"""Presidenas, ah…"


In [34]:
Student_essays = Student_essays.with_columns(
    pl.col('text').map_elements(lambda x: preprocess_text(x))
)
Student_essays

prompt_id,text
i64,list[str]
0,"[""america"", ""love"", … ""question""]"
1,"[""powerful"", ""people"", … ""reason""]"


In [35]:
# # Convert to Pandas DataFrame for easier manipulation
# pandas_df = Student_essays.to_pandas()

# # Tokenize and count word frequencies
# word_counts = (
#     pandas_df['text']
#     .str.split(expand=True)
#     .stack()
#     .groupby([pandas_df['prompt_id'], 0])
#     .count()
#     .unstack(level=0)
#     .fillna(0)
# )
# word_counts

In [36]:
# Extract unique words and count frequencies for each category
Student_unique_words = set()
for text in Student_essays['text']:
    Student_unique_words.update(set(text.to_list()))
    #unique_words.update(text.map_elements(lambda x: set(x.split())))

Student_word_counts = {word: [] for word in Student_unique_words}
for text in Student_essays['text']:
    Student_counts = dict(Counter(text.to_list()))
    for word in Student_unique_words:
        Student_word_counts[word].append(Student_counts.get(word, 0))

In [37]:
# # Convert the dictionary to a list of tuples
# word_tuples = [(word, values[0], values[1]) for word, values in word_counts.items()]

# Student_essays = pl.DataFrame(word_tuples, schema=['Words', '0', '1'])
# Student_essays

In [38]:
# Determine the number of columns based on the length of the lists
#Student_num_columns = len(next(iter(Student_word_counts.values())))
Student_num_columns = 2

# Convert the dictionary to a list of tuples
Student_word_tuples = [(word, *values) for word, values in Student_word_counts.items()]

# Create column names dynamically
Student_column_names = ['Words'] + [f'prompt_id_{i}' for i in range(2)]

# Create a Polars DataFrame
Student_essays = pl.DataFrame(Student_word_tuples, schema=Student_column_names)
Student_essays.sort('Words')

Words,prompt_id_0,prompt_id_1
str,i64,i64
"""aa""",9,2
"""aaron""",2,0
"""abandon""",1,1
"""abandoned""",0,2
"""abandoning""",2,0
"""abandonment""",1,0
"""abbreviated""",0,1
"""abhor""",1,0
"""abide""",3,3


In [39]:
# Storing the total number of words with their respective prompt ID.
Student_total_words = [Student_essays[f'prompt_id_{i}'].sum() for i in range(Student_num_columns)]
Student_total_words

[199455, 160277]

In [40]:
Student_essays_prob = Student_essays.with_columns(
    [pl.col(f'prompt_id_{i}').truediv(Student_total_words[i]).round(10) for i in range(Student_num_columns)]
)
Student_essays_prob

Words,prompt_id_0,prompt_id_1
str,f64,f64
"""poverty""",0.000005,0.0
"""mistaken""",0.0,0.000025
"""building""",0.000316,0.000025
"""care""",0.000406,0.00053
"""rained""",0.00002,0.0
"""struggle""",0.00006,0.000012
"""basket""",0.0,0.000006
"""lying""",0.000005,0.000037
"""argueing""",0.0,0.000006


In [41]:
# # Replace all zeros probabilities to some minimum value
# Student_essays = Student_essays.with_columns(
#     [pl.col(f'prompt_id_{i}').map_elements(lambda x: x if x != 0 else round(1/(Student_total_words[i]+2),10)) for i in range(Student_num_columns)]
# )
# Student_essays

## **Making master dictonary (frequency of each word) LLM dataset**

In [42]:
# Group by 'prompt_id' and concatenate all text values within each group into a single string
LLM_essays = LLM_essays.group_by('prompt_id').agg(
    pl.col('text').str.concat()
)
LLM_essays

prompt_id,text
i64,str
0,"""Title: The Ben…"
1,"""Dear Senator […"


In [43]:
LLM_essays = LLM_essays.with_columns(
    pl.col('text').map_elements(lambda x: preprocess_text(x))
)
LLM_essays

prompt_id,text
i64,list[str]
0,"[""title"", ""benefit"", … ""community""]"
1,"[""dear"", ""senator"", … ""name""]"


In [44]:
# Extract unique words and count frequencies for each category
LLM_unique_words = set()
for text in LLM_essays['text']:
    LLM_unique_words.update(set(text.to_list()))

LLM_word_counts = {word: [] for word in LLM_unique_words}
for text in LLM_essays['text']:
    LLM_counts = dict(Counter(text.to_list()))
    for word in LLM_unique_words:
        LLM_word_counts[word].append(LLM_counts.get(word, 0))

In [45]:
# Determine the number of columns based on the length of the lists
#LLM_num_columns = len(next(iter(LLM_word_counts.values())))
LLM_num_columns = 2

# Convert the dictionary to a list of tuples
LLM_word_tuples = [(word, *values) for word, values in LLM_word_counts.items()]

# Create column names dynamically
LLM_column_names = ['Words'] + [f'prompt_id_{i}' for i in range(LLM_num_columns)]
#print(column_names)

# Create a Polars DataFrame
LLM_essays = pl.DataFrame(LLM_word_tuples, schema=LLM_column_names)
LLM_essays

Words,prompt_id_0,prompt_id_1
str,i64,i64
"""building""",16,2
"""omnipresence""",1,0
"""care""",0,3
"""struggle""",1,0
"""estimated""",1,1
"""revitalizes""",2,0
"""greater""",27,105
"""creeped""",1,0
"""uplift""",1,0


In [46]:
# Storing the total number of words with their respective prompt ID.
LLM_total_words = [LLM_essays[f'prompt_id_{i}'].sum() for i in range(LLM_num_columns)]
LLM_total_words

[94552, 110570]

In [47]:
LLM_essays_prob = LLM_essays.with_columns(
    [pl.col(f'prompt_id_{i}').truediv(LLM_total_words[i]).round(10) for i in range(LLM_num_columns)]
)
LLM_essays_prob

Words,prompt_id_0,prompt_id_1
str,f64,f64
"""building""",0.000169,0.000018
"""omnipresence""",0.000011,0.0
"""care""",0.0,0.000027
"""struggle""",0.000011,0.0
"""estimated""",0.000011,0.000009
"""revitalizes""",0.000021,0.0
"""greater""",0.000286,0.00095
"""creeped""",0.000011,0.0
"""uplift""",0.000011,0.0


In [48]:
# # Replace all zeros probabilities to some minimum value
# LLM_essays = LLM_essays.with_columns(
#     [pl.col(f'prompt_id_{i}').map_elements(lambda x: x if x != 0 else round(1/(LLM_total_words[i]+2),10)) for i in range(LLM_num_columns)]
# )
# LLM_essays

**Total number of words of both Student and LLM with their respective prompt ID.**

In [49]:
Total_words = [Student_total_words[i] + LLM_total_words[i] for i in range(LLM_num_columns)]
Total_words

[294007, 270847]

**Student Probability**

In [50]:
Student_probability = [round(Student_total_words[i]/Total_words[i], 10) for i in range(Student_num_columns)]
Student_probability

[0.6784022149, 0.5917621388]

**LLM (AI Generated) Probability**

In [51]:
LLM_probability = [round(LLM_total_words[i]/Total_words[i], 10) for i in range(LLM_num_columns)]
LLM_probability

[0.3215977851, 0.4082378612]

# **Naive Bayes Classifier**

![Naive Bayes Classifier](https://thatware.co/naive-bayes/)

# **Let's calulate the probability of essay from a train dataset**

In [52]:
random_Student_essay['text'][0]

'The black smoke that leaks out of your car\'s tailpipe is killing you.\n\nWith its toxic chemicals, and the stress that goes along with driving ,it\'s no wonder why our vehicular casualty rate is so high. The answer seems to be clear,there are many advantages in reducing our car use. Not to mention the money saving opportunities and traffic reduction that come along with letting go of our beloved crutch.\n\nThe world has long bin involved in a "love affair" with its cars and other forms of motor transportation, and now in 2015 we are by far, paying the price.\n\nThe dangers to your personal health and the health of the environment are cringe worthy. France, being one of the most polluted ares in Europe. at one point had 147 micrograms of particulate matter in there air. As a result Pairs put a partial driving ban to clear the air and the results where drastic. In just one week the smog cleared up just enough to lift the ban. Passenger cars are responsible for 12 percent of greenhouse 

In [53]:
Student_test_str = preprocess_text(random_Student_essay['text'][0])
Student_test_str

['black',
 'smoke',
 'leak',
 'car',
 'tailpipe',
 'killing',
 'toxic',
 'chemical',
 'stress',
 'go',
 'along',
 'driving',
 'wonder',
 'vehicular',
 'casualty',
 'rate',
 'high',
 'answer',
 'seems',
 'clear',
 'many',
 'advantage',
 'reducing',
 'car',
 'use',
 'mention',
 'money',
 'saving',
 'opportunity',
 'traffic',
 'reduction',
 'come',
 'along',
 'letting',
 'go',
 'beloved',
 'crutch',
 'world',
 'long',
 'bin',
 'involved',
 'love',
 'affair',
 'car',
 'form',
 'motor',
 'transportation',
 'far',
 'paying',
 'price',
 'danger',
 'personal',
 'health',
 'health',
 'environment',
 'cringe',
 'worthy',
 'france',
 'one',
 'polluted',
 'are',
 'europe',
 'one',
 'point',
 'microgram',
 'particulate',
 'matter',
 'air',
 'result',
 'pair',
 'put',
 'partial',
 'driving',
 'ban',
 'clear',
 'air',
 'result',
 'drastic',
 'one',
 'week',
 'smog',
 'cleared',
 'enough',
 'lift',
 'ban',
 'passenger',
 'car',
 'responsible',
 'percent',
 'greenhouse',
 'gas',
 'emission',
 'europe',

In [54]:
prompt_id = random_Student_essay['prompt_id'][0]

In [55]:
random_Student_essay

id,prompt_id,text,generated
str,i64,str,i64
"""c168b452""",0,"""The black smok…",0


The below words are new and not present in data set. So the probabilities of these words will be 0. This zero will ripple through the entire multiplication of all features and gives the resultant probability to zero.

In [56]:
words_not_present = [word for word in Student_test_str if len(LLM_essays.filter(LLM_essays['Words'] == word))==0]
words_not_present

['black',
 'smoke',
 'leak',
 'killing',
 'chemical',
 'casualty',
 'letting',
 'crutch',
 'bin',
 'cringe',
 'are',
 'microgram',
 'pair',
 'week',
 'lift',
 'hard',
 'print',
 'ridding',
 'fantastic',
 'excited',
 'catch',
 'registeres',
 'anymore',
 'acc',
 'bikeing',
 'pudding',
 'welcomed',
 'arm']

# **Laplace Smoother**

One additional issue to be aware of - since naïve Bayes uses the product of feature probabilities conditioned on each class, we run into a serious problem when new data includes a feature value that never occurs for one or more levels of a response class. What results is P(xi|Ck) = 0 for this individual feature and this zero will ripple through the entire multiplication of all features and will always force the posterior probability to be zero for that class.

A solution to this problem involves using the Laplace smoother. The Laplace smoother adds a small number to each of the counts in the frequencies for each feature, which ensures that each feature has a nonzero probability of occuring for each class. Typically, a value of one to two for the Laplace smoother is sufficient, but this is a tuning parameter to incorporate and optimize with cross validation.

In [57]:
test_df

id,prompt_id,text,generated
str,i64,str,i64
"""55ef0196""",1,"""Presidenas, ah…",0
"""aa0b24ef""",1,"""Dear Florida S…",0
"""75c255e3""",1,"""""The founding …",0
"""53393dbd""",1,"""Every 4 years,…",0
"""df69e6ca""",1,"""Dear Senator […",1
"""e7c565f3""",1,"""Dear State Sen…",0
"""3c239e05""",1,"""Dear Florida S…",0
"""9955b24c""",0,"""""But America's…",0
"""66e2c504""",1,"""For years I've…",0


In [58]:
test_df_words = ''
for row in test_df.rows():
    test_df_words += row[2]
test_df_words

'Presidenas, ahey are ahe mosa powerful people in ahe Uniaed Saaaes of America. They are ahe people who make ahe aough calls for ahe masses ao make sure ahe benefias of ahe majoriay are mea because ahe needs of ahe many oua weigh ahe needs of ahe few. Some people ahink ahaa ahe way our presidenas are elecaed aoday are noa very pracaice meahods bua some people do. Presidenas are elecaed using awo processes, ahose processes are popular voae and Elecaoral College. The Elecaoral College is noa very pracaical in my book because ahey don\'a spend aime in ahe places ahaa ahey know ahey are going ao win or noa going ao and Elecaoral College is noa by majoriay.\n\nPresidenas are like supersaars of ahe poliaical world bua ahey don\'a spend any aime in ahe places ahey are know ahey will win or lose. Source 2 saaaes "Candidaaes dona spend ahe aime in saaaes ahey know ahey have no chance of winning." The candidaaes for ahe presidenaial elecaion don"a even ary ao sway some of ahe saaaes ahaa eiaher 

In [59]:
test_df_words_list = preprocess_text(test_df_words)
test_df_words_set = set(test_df_words_list)
test_df_words_set

{'building',
 'omnipresence',
 'care',
 'rained',
 'struggle',
 'lying',
 'analyze',
 'reaffirms',
 'honestly',
 'wait',
 'wheel',
 'relaxing',
 'ease',
 'falling',
 'carved',
 'stretch',
 'agriculture',
 'goodwill',
 'live',
 'combination',
 'drugstore',
 'dearly',
 'socioeconomic',
 'closing',
 'necessarily',
 'terrifying',
 'crisp',
 'define',
 'picking',
 'employes',
 'overrule',
 'writing',
 'relation',
 'benefiting',
 'weird',
 'flawless',
 'behalf',
 'fairly',
 'oriented',
 'intricacy',
 'deliver',
 'integrates',
 'lane',
 'nominating',
 'dismantle',
 'wonder',
 'worldwide',
 'cultural',
 'fought',
 'unpolluted',
 'urging',
 'horrify',
 'consistently',
 'basically',
 'inclusion',
 'preserving',
 'ending',
 'consciousness',
 'voting',
 'defense',
 'realisation',
 'asks',
 'black',
 'upset',
 'cleaning',
 'uncommon',
 'uplifting',
 'annoy',
 'plentiful',
 'recorded',
 'impatient',
 'excellent',
 'considered',
 'bettering',
 'disbelief',
 'matter',
 'outlined',
 'may',
 'inspire',


**Laplace for Student data frame:**

In [60]:
words_in_LLM_not_in_Student = set(LLM_essays['Words'].to_list()) - set(Student_essays['Words'].to_list())
Student_extra_words = [{'Words': word, 'prompt_id_0': 0, 'prompt_id_1': 0} for word in words_in_LLM_not_in_Student]
Student_extra_words_df = pl.DataFrame(Student_extra_words)

#print(Student_extra_words_df)
#print(Student_essays)

final_Student_essays = pl.concat([Student_essays, Student_extra_words_df])
final_Student_essays

Words,prompt_id_0,prompt_id_1
str,i64,i64
"""poverty""",1,0
"""mistaken""",0,4
"""building""",63,4
"""care""",81,85
"""rained""",4,0
"""struggle""",12,2
"""basket""",0,1
"""lying""",1,6
"""argueing""",0,1


In [61]:
words_in_test_df_not_in_Student = test_df_words_set - set(Student_essays['Words'].to_list())
Student_extra_words = [{'Words': word, 'prompt_id_0': 0, 'prompt_id_1': 0} for word in words_in_test_df_not_in_Student]
Student_extra_words_df = pl.DataFrame(Student_extra_words)

#print(Student_extra_words_df)
#print(Student_essays)

final_Student_essays = pl.concat([final_Student_essays, Student_extra_words_df])
final_Student_essays

Words,prompt_id_0,prompt_id_1
str,i64,i64
"""poverty""",1,0
"""mistaken""",0,4
"""building""",63,4
"""care""",81,85
"""rained""",4,0
"""struggle""",12,2
"""basket""",0,1
"""lying""",1,6
"""argueing""",0,1


In [62]:
final_Student_essays = final_Student_essays.with_columns(
    pl.col('prompt_id_0').add(1),
    pl.col('prompt_id_1').add(1)
)
final_Student_essays

Words,prompt_id_0,prompt_id_1
str,i64,i64
"""poverty""",2,1
"""mistaken""",1,5
"""building""",64,5
"""care""",82,86
"""rained""",5,1
"""struggle""",13,3
"""basket""",1,2
"""lying""",2,7
"""argueing""",1,2


In [63]:
# Storing the total number of words with their respective prompt ID.
final_Student_total_words = [final_Student_essays[f'prompt_id_{i}'].sum() for i in range(Student_num_columns)]
final_Student_total_words

[209323, 170145]

In [64]:
Student_essays_prob = final_Student_essays.with_columns(
    [pl.col(f'prompt_id_{i}').truediv(final_Student_total_words[i]).round(10) for i in range(Student_num_columns)]
)
Student_essays_prob

Words,prompt_id_0,prompt_id_1
str,f64,f64
"""poverty""",0.00001,0.000006
"""mistaken""",0.000005,0.000029
"""building""",0.000306,0.000029
"""care""",0.000392,0.000505
"""rained""",0.000024,0.000006
"""struggle""",0.000062,0.000018
"""basket""",0.000005,0.000012
"""lying""",0.00001,0.000041
"""argueing""",0.000005,0.000012


In [65]:
Student_essays_prob = Student_essays_prob.with_columns(
    pl.col('prompt_id_0').mul(10**5),
    pl.col('prompt_id_1').mul(10**5)
)
Student_essays_prob

Words,prompt_id_0,prompt_id_1
str,f64,f64
"""poverty""",0.95546,0.58773
"""mistaken""",0.47773,2.93867
"""building""",30.57476,2.93867
"""care""",39.17391,50.54512
"""rained""",2.38865,0.58773
"""struggle""",6.2105,1.7632
"""basket""",0.47773,1.17547
"""lying""",0.95546,4.11414
"""argueing""",0.47773,1.17547


**Lapalace for LLM data frame**

In [66]:
words_in_Student_not_in_LLM = set(Student_essays['Words'].to_list()) - set(LLM_essays['Words'].to_list())
LLM_extra_words = [{'Words': word, 'prompt_id_0': 0, 'prompt_id_1': 0} for word in words_in_Student_not_in_LLM]
LLM_extra_words_df = pl.DataFrame(LLM_extra_words)

final_LLM_essays = pl.concat([LLM_essays, LLM_extra_words_df])
final_LLM_essays

Words,prompt_id_0,prompt_id_1
str,i64,i64
"""building""",16,2
"""omnipresence""",1,0
"""care""",0,3
"""struggle""",1,0
"""estimated""",1,1
"""revitalizes""",2,0
"""greater""",27,105
"""creeped""",1,0
"""uplift""",1,0


In [67]:
words_in_test_df_not_in_LLM = test_df_words_set - set(LLM_essays['Words'].to_list())
LLM_extra_words = [{'Words': word, 'prompt_id_0': 0, 'prompt_id_1': 0} for word in words_in_test_df_not_in_LLM]
LLM_extra_words_df = pl.DataFrame(LLM_extra_words)

final_LLM_essays = pl.concat([final_LLM_essays, LLM_extra_words_df])
final_LLM_essays

Words,prompt_id_0,prompt_id_1
str,i64,i64
"""building""",16,2
"""omnipresence""",1,0
"""care""",0,3
"""struggle""",1,0
"""estimated""",1,1
"""revitalizes""",2,0
"""greater""",27,105
"""creeped""",1,0
"""uplift""",1,0


In [68]:
final_LLM_essays = final_LLM_essays.with_columns(
    pl.col('prompt_id_0').add(1),
    pl.col('prompt_id_1').add(1)
)
final_LLM_essays

Words,prompt_id_0,prompt_id_1
str,i64,i64
"""building""",17,3
"""omnipresence""",2,1
"""care""",1,4
"""struggle""",2,1
"""estimated""",2,2
"""revitalizes""",3,1
"""greater""",28,106
"""creeped""",2,1
"""uplift""",2,1


In [69]:
# Storing the total number of words with their respective prompt ID.
final_LLM_total_words = [final_LLM_essays[f'prompt_id_{i}'].sum() for i in range(LLM_num_columns)]
final_LLM_total_words

[105933, 121951]

In [70]:
LLM_essays_prob = final_LLM_essays.with_columns(
    [pl.col(f'prompt_id_{i}').truediv(final_LLM_total_words[i]).round(10) for i in range(LLM_num_columns)]
)
LLM_essays_prob

Words,prompt_id_0,prompt_id_1
str,f64,f64
"""building""",0.00016,0.0000246
"""omnipresence""",0.000019,0.0000082
"""care""",0.000009,0.000033
"""struggle""",0.000019,0.0000082
"""estimated""",0.000019,0.0000164
"""revitalizes""",0.000028,0.0000082
"""greater""",0.000264,0.000869
"""creeped""",0.000019,0.0000082
"""uplift""",0.000019,0.0000082


In [71]:
LLM_essays_prob = LLM_essays_prob.with_columns(
    pl.col('prompt_id_0').mul(10**5),
    pl.col('prompt_id_1').mul(10**5)
)
LLM_essays_prob

Words,prompt_id_0,prompt_id_1
str,f64,f64
"""building""",16.04788,2.46
"""omnipresence""",1.88799,0.82
"""care""",0.94399,3.28001
"""struggle""",1.88799,0.82
"""estimated""",1.88799,1.64
"""revitalizes""",2.83198,0.82
"""greater""",26.4318,86.92016
"""creeped""",1.88799,0.82
"""uplift""",1.88799,0.82


Smoothing is done. Let's calculate the probability.

So now every words are present in data frame.

In [72]:
words_not_present = [word for word in Student_test_str if len(LLM_essays_prob.filter(LLM_essays_prob['Words'] == word))==0]
words_not_present

[]

In [73]:
from decimal import Decimal, getcontext, ROUND_HALF_UP

# Set the precision for the Decimal module
getcontext().prec = 20

In [74]:
#LLM_essays_prob.filter(LLM_essays_prob['Words'] == 'jimmy')[0,1]
numerator = Decimal(1)
for word in Student_test_str:
    numerator = numerator * Decimal(LLM_essays_prob.filter(LLM_essays_prob['Words'] == word)[0,1])
numerator = numerator * Decimal(LLM_probability[0])
#numerator = numerator * 10**30
numerator

Decimal('5.2810013409432117812E+416')

In [75]:
denominator = Decimal(1)
for word in Student_test_str:
    denominator = denominator * Decimal(Student_essays_prob.filter(Student_essays_prob['Words'] == word)[0,1])
denominator = denominator * Decimal(Student_probability[0])
#denominator = denominator * 10**30
print(denominator)
denominator = numerator + denominator
denominator

3.3389362288541853953E+493


Decimal('3.3389362288541853953E+493')

In [76]:
final_probability = numerator/denominator
final_probability

Decimal('1.5816418700381948101E-77')

So it is equivalent to Zero, which is correct answer as it not LLM generated essay but Student written essay.

In [77]:
def calculate_probability(test_str, prompt_id):
    numerator = Decimal(1)
    for word in test_str:
        filter_df = LLM_essays_prob.filter(LLM_essays_prob['Words'] == word)
        if not filter_df.is_empty():
            numerator = numerator * Decimal(filter_df[0,prompt_id+1])
        else:
            print(word)
    numerator = numerator * Decimal(LLM_probability[prompt_id])
    #numerator = numerator * 10**30
    
    denominator = Decimal(1)
    for word in test_str:
        filter_df = Student_essays_prob.filter(Student_essays_prob['Words'] == word)
        if not filter_df.is_empty():
            denominator = denominator * Decimal(filter_df[0,prompt_id+1])
        else:
            print(word)
    denominator = denominator * Decimal(Student_probability[prompt_id])
    #denominator = denominator * 10**30
    denominator = numerator + denominator
    
    final_probability = numerator/denominator
    
    # Round to three decimal places using ROUND_HALF_UP rounding method
    rounded_final_probability = final_probability.quantize(Decimal('1.0'), rounding=ROUND_HALF_UP)
    return float(rounded_final_probability)

In [78]:
submittion_df = pl.read_csv('/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv')
submittion_df

id,generated
str,f64
"""0000aaaa""",0.1
"""1111bbbb""",0.9
"""2222cccc""",0.4


In [79]:
submittion_list = []

In [80]:
for rows in test_df.rows():
    preproceesed_str = preprocess_text(rows[2])
    probability = calculate_probability(preproceesed_str, rows[1])
    submittion_list.append((rows[0],probability))
submittion_list

humankind
humankind
fuji
fuji


[('55ef0196', 0.0),
 ('aa0b24ef', 0.0),
 ('75c255e3', 0.0),
 ('53393dbd', 0.0),
 ('df69e6ca', 1.0),
 ('e7c565f3', 0.0),
 ('3c239e05', 0.0),
 ('9955b24c', 0.0),
 ('66e2c504', 0.0),
 ('15f7ea58', 0.0),
 ('2cd15a38', 0.0),
 ('2ac6b992', 0.0),
 ('020a5d6d', 0.0),
 ('a8f57f7a', 0.0),
 ('8390d932', 0.0),
 ('68ad0312', 1.0),
 ('eff68a37', 0.0),
 ('c8b89dd4', 0.0),
 ('3f024d8f', 0.0),
 ('4264a40f', 0.0),
 ('bad69fc1', 1.0),
 ('b4e6f8e6', 1.0),
 ('c71d940b', 0.0),
 ('0a13d187', 0.0),
 ('bc749273', 0.0),
 ('99a76936', 0.0),
 ('aed17226', 0.0),
 ('8085e801', 0.0),
 ('7e5e9ffb', 0.0),
 ('b3853076', 0.0),
 ('94044ae2', 0.0),
 ('d4fcfd4a', 1.0),
 ('175003c6', 0.0),
 ('267cc695', 0.0),
 ('eb393a7d', 0.0),
 ('d4b3d8d2', 0.0),
 ('9f345348', 0.0),
 ('a6d8aa63', 0.0),
 ('79255ae0', 0.0),
 ('15c27f02', 1.0),
 ('44558f42', 0.0),
 ('72de1467', 0.0),
 ('42537329', 0.0),
 ('d429f032', 1.0),
 ('d01b5128', 1.0),
 ('81acef36', 0.0),
 ('b698e899', 0.0),
 ('0d1a2bb2', 1.0),
 ('74a42ef1', 0.0),
 ('e652b5f5', 1.0),


In [81]:
submission_column_names = ['id', 'generated1']
submission = pl.DataFrame(submittion_list, schema=submission_column_names)

In [82]:
submission

id,generated1
str,f64
"""55ef0196""",0.0
"""aa0b24ef""",0.0
"""75c255e3""",0.0
"""53393dbd""",0.0
"""df69e6ca""",1.0
"""e7c565f3""",0.0
"""3c239e05""",0.0
"""9955b24c""",0.0
"""66e2c504""",0.0


In [83]:
#submission.write_csv('submission.csv')

In [84]:
test_df

id,prompt_id,text,generated
str,i64,str,i64
"""55ef0196""",1,"""Presidenas, ah…",0
"""aa0b24ef""",1,"""Dear Florida S…",0
"""75c255e3""",1,"""""The founding …",0
"""53393dbd""",1,"""Every 4 years,…",0
"""df69e6ca""",1,"""Dear Senator […",1
"""e7c565f3""",1,"""Dear State Sen…",0
"""3c239e05""",1,"""Dear Florida S…",0
"""9955b24c""",0,"""""But America's…",0
"""66e2c504""",1,"""For years I've…",0


In [85]:
joined_df = submission.join(test_df, on='id', how='inner')
joined_df

id,generated1,prompt_id,text,generated
str,f64,i64,str,i64
"""55ef0196""",0.0,1,"""Presidenas, ah…",0
"""aa0b24ef""",0.0,1,"""Dear Florida S…",0
"""75c255e3""",0.0,1,"""""The founding …",0
"""53393dbd""",0.0,1,"""Every 4 years,…",0
"""df69e6ca""",1.0,1,"""Dear Senator […",1
"""e7c565f3""",0.0,1,"""Dear State Sen…",0
"""3c239e05""",0.0,1,"""Dear Florida S…",0
"""9955b24c""",0.0,0,"""""But America's…",0
"""66e2c504""",0.0,1,"""For years I've…",0


In [86]:
miss_count = 0
for rows in joined_df.rows():
    if rows[1] != float(rows[4]):
        miss_count += 1
miss_count

1

In [87]:
accuracy = ((len(test_df)-miss_count)/len(test_df))*100
accuracy

99.76190476190476